In [85]:
import torch
import torch.nn as nn
from torch.nn import functional as F

text = open("input.txt", "r", encoding="utf-8").read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: s for s, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda d: "".join([itos[c] for c in d])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(len(data) * 0.9)
train = data[:n]
val = data[n:]
block_size = 8
batch_size = 4


def get_batch(split):
    data = train if split == "train" else val
    ix = torch.randint(len(data) - block_size, (batch_size,))  # get 4 random ints
    x = torch.stack([data[i : i + block_size] for i in ix])  # get context
    y = torch.stack([data[i + 1 : i + 1 + block_size] for i in ix])  # get preds
    return x, y



class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        B, T, C = logits.shape
        logits = logits.view(B * T, C)
        if targets is not None:
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


model = BigramLanguageModel(vocab_size)

In [86]:
optim = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [96]:
max_iter = 3000
eval_iters = 200

@torch.no_grad()
def compute_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range (eval_iters):
            X, y = get_batch(split)
            logits, loss = model(X, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [ ]:
for i in range(max_iter):
    if i % eval_iters == 0:
        loss = compute_loss()
        print(f'train loss = {loss["train"]} val loss = {loss["val"]}')
    xb, yb = get_batch('train')
    logits,loss = model(xb, yb)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()

train loss = 4.5705976486206055 val loss = 4.574448585510254
train loss = 4.573195457458496 val loss = 4.577404975891113
train loss = 4.5689263343811035 val loss = 4.57314920425415
train loss = 4.571704387664795 val loss = 4.574051380157471
